In [1]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval
# d808a1c335762ae67bc9777128840c0072a22f24

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 629.7 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=a924149c43aedcae29db7a4ca0e1d8d8ec309a4484ef31446cd91e94212ba552
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.27.2
    Uninstalling accelerate-0.27.2:
      Successfully uninstalled accelerate-0.27.2


In [3]:
!pip install transformers[torch]

**Loading Dataset**

In [4]:
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
text_column_name = "tokens"
label_column_name = "ner_tags"

In [6]:
print(raw_datasets['test'][1])

{'tokens': ['शाहरुख', 'खान', 'इन', 'दिनों', 'अपनी', 'आगामी', 'फिल्म', 'जीरो', 'की', 'शूटिंग', 'में', 'व्यस्त', 'हैं', '।'], 'ner_tags': [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [7]:
features = raw_datasets["train"].features

**Mapping of Classes to Integers**

In [8]:
label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)

{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


**Loading INDIC-NER**

In [9]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/indicNER', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indicNER")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indicNER', num_labels=num_labels )

2024-03-13 07:18:53.442640: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 07:18:53.442765: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 07:18:53.609205: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [10]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

**Training Dataset**

In [11]:
train_dataset = raw_datasets["train"]
train_dataset=train_dataset.select(range(20000))
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset #0:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #1:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #2:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #3:   0%|          | 0/5 [00:00<?, ?ba/s]

**Validation Dataset**

In [12]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset #0:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #2:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #1:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #3:   0%|          | 0/4 [00:00<?, ?ba/s]

In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [14]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

In [81]:
# args=TrainingArguments(output_dir='output_dir',max_steps=5)
# args=TrainingArguments(
#     output_dir='output_dir',
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3)
batch_size=8
args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    evaluation_strategy = "epoch",
    learning_rate=4e-5)

In [82]:
# Initialize our Trainer
# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)
# args.metric_for_best_model = "f1"
# args.load_best_model_at_end = True
# args.evaluation_strategy = IntervalStrategy.STEPS
# args.eval_steps = args.save_steps
# args.greater_is_better = True

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[early_stopping_callback],
    args=args,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [83]:
trainer.args

TrainingArguments(
_n_gpu=2,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_

**Training the Model**

In [84]:
train_result = trainer.train()
metrics = train_result.metrics

Epoch,Training Loss,Validation Loss,Loc Precision,Loc Recall,Loc F1,Loc Number,Org Precision,Org Recall,Org F1,Org Number,Per Precision,Per Recall,Per F1,Per Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.073600,0.223459,0.807972,0.837560,0.822500,10213,0.664523,0.686389,0.675279,9786,0.796529,0.820874,0.808519,10568,0.758121,0.783394,0.770550,0.944361
2,0.054000,0.244642,0.810656,0.831293,0.820845,10213,0.659564,0.676885,0.668112,9786,0.788212,0.821253,0.804393,10568,0.754686,0.778388,0.766354,0.943884
3,0.037400,0.269991,0.805070,0.830216,0.817450,10213,0.648524,0.675761,0.661863,9786,0.788395,0.815102,0.801526,10568,0.748886,0.775542,0.761981,0.943076


Checkpoint destination directory output_dir/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Checkpoint destination directory output_dir/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Checkpoint destination directory output_dir/checkpoint-1500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/opt/conda/l

**Validation Metrics(Precision,Recall and F1 Score)**

In [85]:
metrics = trainer.evaluate()

trainer.log_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        3.0
  eval_LOC_f1             =     0.8174
  eval_LOC_number         =      10213
  eval_LOC_precision      =     0.8051
  eval_LOC_recall         =     0.8302
  eval_ORG_f1             =     0.6619
  eval_ORG_number         =       9786
  eval_ORG_precision      =     0.6485
  eval_ORG_recall         =     0.6758
  eval_PER_f1             =     0.8015
  eval_PER_number         =      10568
  eval_PER_precision      =     0.7884
  eval_PER_recall         =     0.8151
  eval_loss               =       0.27
  eval_overall_accuracy   =     0.9431
  eval_overall_f1         =      0.762
  eval_overall_precision  =     0.7489
  eval_overall_recall     =     0.7755
  eval_runtime            = 0:04:48.79
  eval_samples_per_second =     46.608
  eval_steps_per_second   =       2.33


In [86]:
model.save_pretrained("NER_model")

In [87]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [25]:
# !zip -r file.zip /kaggle/working/NER_model

  adding: kaggle/working/NER_model/ (stored 0%)
  adding: kaggle/working/NER_model/model.safetensors (deflated 7%)
  adding: kaggle/working/NER_model/config.json (deflated 54%)


In [88]:
tokenized_test_set = {}

tokenized_test_set = raw_datasets['test'].map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset of language {0}".format(lang),
)

Running tokenizer on test dataset of language hi #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [89]:
final_metrics = {}


predictions, labels, metrics = trainer.predict(tokenized_test_set, metric_key_prefix=lang)

lang_specific_results = {}
for key in metrics:
  if 'overall_precision' in key:
    lang_specific_results['Precision'] = metrics[key]
  elif 'overall_recall' in key:
    lang_specific_results['Recall'] = metrics[key]
  elif 'overall_f1' in key:
    lang_specific_results['F1'] = metrics[key]
final_metrics = lang_specific_results

**Test_Set Metrics(Precision,Recall and F1-Score)**

In [90]:
print(final_metrics)

{'Precision': 0.7525374577090382, 'Recall': 0.807153965785381, 'F1': 0.7788894447223612}


***Q.4***

In [91]:
model.save_pretrained("ner_model")

In [92]:
model = AutoModelForTokenClassification.from_pretrained("NER_model")

**Dataset of 25 sentences**

In [93]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}
print(id2label)
print(label2id)

{'0': 'O', '1': 'B-PER', '2': 'I-PER', '3': 'B-ORG', '4': 'I-ORG', '5': 'B-LOC', '6': 'I-LOC'}
{'O': '0', 'B-PER': '1', 'I-PER': '2', 'B-ORG': '3', 'I-ORG': '4', 'B-LOC': '5', 'I-LOC': '6'}


In [94]:
import json
config=json.load(open("/kaggle/working/NER_model/config.json"))

In [95]:
config["id2label"] = id2label
config["label2id"] = label2id

In [96]:
json.dump(config, open("/kaggle/working/NER_model/config.json","w"))

In [97]:
model = AutoModelForTokenClassification.from_pretrained("NER_model")

In [98]:
from datasets import Dataset
ds={'tokens':[['मिशेल', 'ने', 'कहा', 'कि', 'मैं', 'यहां', 'सबसे', 'पहले', 'और', 'सबसे', 'आगे', 'हूं', ' ,', 'क्योंकि', 'हमें', 'राष्ट्रपति', 'पद', 'के', 'लिए', 'हमारी', 'दोस्त', 'हिलेरी', ' से', 'ज्यादा', 'योग्य', 'और', 'तैयार', 'उम्मीदवार', 'कभी', 'नहीं', 'मिला', '।'], ['उस', 'ताम्बे', 'की', 'परत', 'पे', 'जमे', 'परतों', 'के', 'जो', 'रंग', 'हैं', ','], ['सरकार', 'ने', 'हमें', 'अपने', 'हाल', 'पर', 'मरने', 'के', 'लिए', 'छोड़', 'दिया', 'है', '।'], ['इतना', 'ही', 'नहीं', 'उसने', 'पिछले', '2', 'वर्षों', 'में', '6', 'अन्य', 'महिलाओं', 'केसाथ', 'यौन', 'उत्पीड़न', 'भी', 'किया', '।'], ['राजे', 'छह', 'राज्यों', 'के', 'मुख्यमंत्रियों', 'के', 'साथ', 'नीति', 'आयोग', 'की', 'बैठक', 'में', 'भाग', 'लेने', 'दिल्ली', 'आईं', 'थी', '।'], ['क्रांतिदूत', 'आजादी', '/', 'विभाजन', 'पूर्व', 'के', 'वे', 'पन्द्रह', 'दिन', ' -', '१४', 'अगस्त', ',', '१९४७', '-', 'सुबह', '9', 'बजे', 'बना', 'पाकिस्तान', 'और', 'मध्यरात्रि', '12', 'बजे', 'भारत', 'हुआ', 'आजाद', ' -', 'प्रशांत', 'पोळ'], ['मप्र', 'हाईकोर्ट', 'ने', 'भोपाल', 'के', 'लिए', 'निर्धारित', 'बाजार', 'मूल्य', 'की', 'गाइड-लाइन', 'के', 'उस', 'प्रावधान', 'पर', 'पूर्व', 'में', 'लगायी', 'गई', 'रोक', 'बरकरार', 'रखी', 'है', ',', 'जिसके', 'तहत', 'फ्लैटों', 'की', 'बाजार', 'मूल्य', 'की', 'गणना', 'बिल्ट-अप', 'एरिया', 'व', 'सामान्य', 'सुविधाओं', 'को', 'शामिल', 'करते', 'हुए', 'की', 'जानी', 'थी', '।'], ['जब', 'मोदी', 'मजबूत', 'होंगे', 'तो', 'देश', 'भी', 'और', 'मजबूत', 'होगा', '।'], ['इस', 'आम', 'चुनाव', 'में', 'भाजपा', 'नेता', 'सतीश', 'कुमार', 'गौतम', 'को', 'सबसे', 'अधिक', '6', 'लाख', '56', 'हजार', '215', 'वोट', 'प्राप्त', 'हुए', '।'], ['.4)', 'देश', 'में', 'सभी', 'प्रकार', 'के', 'छोटे-बड़े', 'कृषि', ',', 'सेवा', ',', 'व्यापार', ',', 'उद्योग', 'का', 'पंजीकरण', 'तुरंत', 'अनिवार्य', 'होना', 'चाहिए', ',', 'साथ', 'उनको', 'रु', '1000', 'से', 'ऊपर', 'के', 'लेन', 'देन', 'को', 'ऑन', 'लाइन/', 'इलेक्ट्रॉनिक', 'माध्यम', 'से', 'करना', 'अनिवार्य', 'होना', 'चाहिये।'], ['निर्भया', 'के', 'परिजनों', 'ने', 'आवेदन', 'किया', 'है', 'कि', 'उन्हें', 'फांसी', 'दिए', 'जाने', 'के', 'समय', 'उस', 'स्थान', 'पर', 'मौजूद', 'रहने', 'की', 'इजाजत', 'दी', 'जाए।'], ['कच्चे', 'माल', 'की', 'लागत', 'राजस्व', 'का', 'तकरीबन', '60', 'प्रतिशत', 'होती', 'है', 'और', 'दिसंबर', 'से', 'अप्रैल', 'के', 'बीच', 'इसमें', '25', 'प्रतिशत', 'तक', 'का', 'इजाफा', 'हो', 'चुका', 'है।'], ['उनका', 'अखबार', 'बंद', 'भले', 'हो', 'गया', 'पर', 'भाषा', 'पर', 'सरकारी', 'मोहर', 'लग', 'चुकी', 'है।'], ['इस', 'पर', 'सेना', 'की', '14', 'राष्ट्रीय', 'राइफल्स', ',', 'जम्मू-कश्मीर', 'पुलिस', 'के', 'स्पेशल', 'ऑपरेशन', 'ग्रुप', 'और', 'केंद्रीय', 'रिजर्व', 'पुलिस', 'बल', 'के', 'जवानों', 'ने', 'जंगलों', 'की', 'घेराबंदी', 'कर', 'तलाशी', 'अभियान', 'शुरू', 'किया।'], ['.कट्टर', 'सेक्स', ',', 'बिग', 'डिक', ',', 'बड़ी', 'डिक्स', ',', 'उसे', 'बिल्ली', 'गड़बड़', 'हो', 'रही', 'है', ',', 'बड़े', 'स्तन', ',', 'बिल्ली', ',', 'उसे', 'बिल्ली', 'कुचल', ',', 'उसे', 'बिल्ली', 'पर', ',', 'उसे', 'बिल्ली', 'गड़बड़', 'हो', ',', 'get', 'ur', 'pussy', 'wet', 'porn', ',', 'सभी', 'उसे', 'pussi', 'अश्लील', ',', 'लड़की', 'अपने', 'पैर', 'की', 'उंगलियों', 'चूसना', 'हो', 'जाता', 'है।'], ['आज', 'की', 'तारीख', 'है', '10', 'अगस्त', '2018', 'तथा', 'आज', 'इस', 'वर्ष', 'का', '222वां', 'दिन', 'है।', 'इन', '222', 'दिनों', 'में', 'प्रारंग', 'अपने', 'परिवार', 'के', 'जौनपुरवासियों', 'तक', '208', 'लेख', 'पहुंचा', 'चुका', 'है', 'तथा', 'यह', 'लेख', '209वां', 'लेख', 'होगा।'], ['वहीं', 'प्रबंधन', 'समिति', 'कुसुम', 'चाचान', 'कहतीं', 'हैं', 'कि', 'मरीजों', 'एवं', 'उनके', 'स्वजनों', 'को', 'निशुल्क', 'भोजन', 'कराकर', 'सुख', 'की', 'अनुभूति', 'होती', 'है।'], ['स्थिर', 'दर', 'को', 'पांच', 'से', '10', 'साल', 'के', 'लिए', 'निश्चित', 'रखा', 'जा', 'सकता', 'है', 'और', 'उसके', 'बाद', 'वह', 'परिवर्तनशील', 'ब्याज', 'दर', 'की', 'श्रेणी', 'में', 'आ', 'जाएगा।'], ['इस', 'सर्वेक्षण', 'में', 'छोटे', 'स्तर', 'की', '49', 'फीसदी', 'इकाइयां', ',', '35', 'फीसदी', 'मझोली', 'इकाइयां', 'और', '16', 'फीसदी', 'बड़ी', 'इकाइयां', 'शामिल', 'हुईं।'], ['सामाजिक', 'संस्थाओं', 'व', 'आप', 'कार्यकर्ताओं', 'ने', 'नगरपालिका', 'के', 'खिलाफ', 'जमकर', 'नारेबाजी', 'की', '।'], ['संघ', 'के', 'प्रांतीय', 'सचिव', 'एवं', 'कोषाध्यक्ष', 'रणदीप', 'आर्य', 'ने', 'बताया', 'कि', 'बिजली', 'विभाग', 'में', 'कभी', 'ट्रांसफार्मर', 'दिलवाने', 'के', 'नाम', 'पर', 'और', 'कभी', 'तेल', 'व', 'केबल', 'के', 'नाम', 'पर', 'अतिरिक्त', 'शुल्क', 'मांगा', 'जाता', 'है', 'और', 'किसान', 'को', 'मजबूरीवश', 'यह', 'शुल्क', 'देना', 'पड़ता', 'है', '।'], ['मुरादाबाद', '/', 'सिपाही', 'ने', 'कैंटीन', 'में', 'खड़े', 'लोगों', 'को', 'डंडों', 'से', 'पीटा', ',', 'रिश्वत', 'भी', 'मांगी'], ['जी', 'हां', 'विश्व', 'का', 'पहला', 'वजाइना', 'म्यूजियम', 'लंदन', 'में', 'खुलने', 'जा', 'रहा', 'है।'], ['टॉयलेट', 'एक', 'कथा', '', 'बिलासपुर', 'की', 'लड़की', 'ने', 'शादी', 'से', 'इनकार', 'किया', 'तब', 'युवक', 'ने', 'बनवाया', 'टॉयलेट', ',', 'दिल्ली', 'की', 'टीम', 'ने', 'द', '|'], ['राजीव', 'गांधी', 'उर्फ', 'बॉबी', 'ने', 'बताया', 'कि', 'विरोध', 'पर', 'नगर', 'निगम', 'ने', 'टावर', 'की', 'स्वीकृति', 'रद्द', 'कर', 'दी', 'थी', '।']]
    ,'ner_tags':[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 5, 0, 0, 0], [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 5, 0, 0, 0, 1, 2], [5, 6, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 3, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 3, 4, 4, 0, 5, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0], [0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0], [1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}
    
ds = Dataset.from_dict(ds)


In [99]:
tokenized_test_set = {}

tokenized_test_set = ds.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=32,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset of language {0}".format(lang),
)

Running tokenizer on test dataset of language hi #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #3:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #4:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Running tokenizer on test dataset of language hi #21:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #20:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #22:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #23:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #24:   0%|          | 0/1 [00:00<?, ?ba/s]

In [100]:
final_metrics = {}


predictions, labels, metrics = trainer.predict(tokenized_test_set, metric_key_prefix=lang)

lang_specific_results = {}
for key in metrics:
  if 'overall_precision' in key:
    lang_specific_results['Precision'] = metrics[key]
  elif 'overall_recall' in key:
    lang_specific_results['Recall'] = metrics[key]
  elif 'overall_f1' in key:
    lang_specific_results['F1'] = metrics[key]
final_metrics = lang_specific_results

**25 Sentences Metric(Precision,Recall,F1-Score)**

In [101]:
print(final_metrics)

{'Precision': 0.5555555555555556, 'Recall': 0.625, 'F1': 0.5882352941176471}


In [102]:
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)
    
    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]
    
    predicted_labels = []
    
    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]
    
    return predicted_labels

**Tokenization of 25 Sentences**

In [103]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")


sentences=['मिशेल ने कहा कि मैं यहां सबसे पहले और सबसे आगे हूं, क्योंकि हमें राष्ट्रपति पद के लिए हमारी दोस्त हिलेरी से ज्यादा योग्य और तैयार उम्मीदवार कभी नहीं मिला।',
           'उस ताम्बे की परत पे जमे परतों के जो रंग हैं,',
           'सरकार ने हमें अपने हाल पर मरने के लिए छोड़ दिया है।',
           'इतना ही नहीं उसने पिछले 2 वर्षों में 6 अन्य महिलाओं केसाथ यौन उत्पीड़न भी किया।',
           'राजे छह राज्यों के मुख्यमंत्रियों के साथ नीति आयोग की बैठक में भाग लेने दिल्ली आईं थी।',
           'क्रांतिदूत: आजादी / विभाजन पूर्व के वे पन्द्रह दिन - १४ अगस्त, १९४७ - सुबह 9 बजे बना पाकिस्तान और मध्यरात्रि 12 बजे भारत हुआ आजाद - प्रशांत पोळ',
           'मप्र हाईकोर्ट ने भोपाल के लिए निर्धारित बाजार मूल्य की गाइड-लाइन के उस प्रावधान पर पूर्व में लगायी गई रोक बरकरार रखी है, जिसके तहत फ्लैटों की बाजार मूल्य की गणना बिल्ट-अप एरिया व सामान्य सुविधाओं को शामिल करते हुए की जानी थी।',
           'जब मोदी मजबूत होंगे तो देश भी और मजबूत होगा।',
           'इस आम चुनाव में भाजपा नेता सतीश कुमार गौतम को सबसे अधिक 6 लाख 56 हजार 215 वोट प्राप्त हुए.',
           '4) देश में सभी प्रकार के छोटे-बड़े कृषि, सेवा, व्यपार,उद्योग का पंजीकरण तुरंत अनिवार्य होना चाहिए, साथ उनको रु 1000 से ऊपर के लेन देन को ऑन लाइन/ इलेक्ट्रॉनिक माध्यम से करना अनिवार्य होना चाहिये।',
           'निर्भया के परिजनों ने आवेदन किया है कि उन्हें फांसी दिए जाने के समय उस स्थान पर मौजूद रहने की इजाजत दी जाए।',
           'कच्चे माल की लागत राजस्व का तकरीबन 60 प्रतिशत होती है और दिसंबर से अप्रैल के बीच इसमें 25 प्रतिशत तक का इजाफा हो चुका है।',
           'उनका अखबार बंद भले हो गया पर भाषा पर सरकारी मोहर लग चुकी है।',
           'इस पर सेना की 14 राष्ट्रीय राइफल्स, जम्मू-कश्मीर पुलिस के स्पेशल ऑपरेशन ग्रुप और केंद्रीय रिजर्व पुलिस बल के जवानों ने जंगलों की घेराबंदी कर तलाशी अभियान शुरू किया।',
           'कट्टर सेक्स, बिग डिक, बड़ी डिक्स, उसे बिल्ली गड़बड़ हो रही है, बड़े स्तन, बिल्ली, उसे बिल्ली कुचल, उसे बिल्ली पर, उसे बिल्ली गड़बड़ हो, get ur pussy wet porn, सभी उसे pussi अश्लील, लड़की अपने पैर की उंगलियों चूसना हो जाता है',
           'आज की तारीख है 10 अगस्त 2018 तथा आज इस वर्ष का 222वां दिन है। इन 222 दिनों में प्रारंग अपने परिवार के जौनपुरवासियों तक 208 लेख पहुंचा चुका है तथा यह लेख 209वां लेख होगा।',
           'वहीं प्रबंधन समिति कुसुम चाचान कहतीं हैं कि मरीजों एवं उनके स्वजनों को निशुल्क भोजन कराकर सुख की अनुभूति होती है।',
           'स्थिर दर को पांच से 10 साल के लिए निश्चित रखा जा सकता है और उसके बाद वह परिवर्तनशील ब्याज दर की श्रेणी में आ जाएगा।',
           'इस सर्वेक्षण में छोटे स्तर की 49 फीसदी इकाइयां, 35 फीसदी मझोली इकाइयां और 16 फीसदी बड़ी इकाइयां शामिल हुईं।',
           'सामाजिक संस्थाओं व आप कार्यकर्ताओं ने नगरपालिका के खिलाफ जमकर नारेबाजी की।',
        'संघ के प्रांतीय सचिव एवं कोषाध्यक्ष रणदीप आर्य ने बताया कि बिजली विभाग में कभी ट्रांसफार्मर दिलवाने के नाम पर और कभी तेल व केबल के नाम पर अतिरिक्त शुल्क मांगा जाता है और किसान को मजबूरीवश यह शुल्क देना पड़ता है।',
           'मुरादाबाद / सिपाही ने कैंटीन में खड़े लोगों को डंडों से पीटा, रिश्वत भी मांगी',
           'जी हां विश्व का पहला वजाइना म्यूजियम लंदन में खुलने जा रहा है।',
           'टॉयलेट एक कथा: बिलासपुर की लड़की ने शादी से इनकार किया तब युवक ने बनवाया टॉयलेट, दिल्ली की टीम ने द  |',
           'राजीव गांधी उर्फ बॉबी ने बताया कि विरोध पर नगर निगम ने टावर की स्वीकृति रद्द कर दी थी।']
Indic_NER=[]
for sentence in sentences:
    predicted_labels = get_predictions(sentence=sentence, 
                                   tokenizer=tokenizer,
                                   model=model
                                   )
    Indic_NER.append(predicted_labels)

**Predicted Labels for 25 Sentences**

In [104]:
print(Indic_NER)

[['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O'], ['O', 'O', 'B-LOC', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'B-PER', 'O'], ['B-ORG', 'I-ORG', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O

**Ground_truth and Predicted Labels**

In [5]:
ground_truth='''
मिशेल:B-PER;ने:O;कहा:O;कि:O;मैं:O;यहां:O;सबसे:O;पहले:O;और:O;सबसे:O;आगे:O;हूं:O; ,:O;क्योंकि:O;हमें:O;राष्ट्रपति:B-MISC;पद:O;के:O;लिए:O;हमारी:O;दोस्त:O;हिलेरी:B-PER; से:O;ज्यादा:O;योग्य:O;और:O;तैयार:O;उम्मीदवार:O;कभी:O;नहीं:O;मिला:O;।:O;


उस:O;ताम्बे:B-MISC;की:O;परत:O;पे:O;जमे:O;परतों:O;के:O;जो:O;रंग:O;हैं:O;,:O


सरकार:O;ने:O;हमें:O;अपने:O;हाल:O;पर:O;मरने:O;के:O;लिए:O;छोड़:O;दिया:O;है:O;।:O


इतना:O;ही:O;नहीं:O;उसने:O;पिछले:O;2:B-MISC;वर्षों:O;में:O;6:B-MISC;अन्य:O;महिलाओं:O;केसाथ:O;यौन:O;उत्पीड़न:O;भी:O;किया:O;।:O


राजे:B-PER;छह:B-MISC;राज्यों:O;के:O;मुख्यमंत्रियों:O;के:O;साथ:O;नीति:B-ORG;आयोग:I-ORG;की:O;बैठक:O;में:O;भाग:O;लेने:O;दिल्ली:B-LOC;आईं:O;थी:O;।:O


क्रांतिदूत:B-ORG;आजादी:O;/:O;विभाजन:O;पूर्व:O;के:O;वे:O;पन्द्रह:B-MISC;दिन:O; -:O;१४:B-MISC;अगस्त:I-MISC;,:O;१९४७:I-MISC;-:O;सुबह:O;9:B-MISC;बजे:O;बना:O;पाकिस्तान:B-LOC;और:O;मध्यरात्रि:O;12:B-MISC;बजे:O;भारत:B-LOC;हुआ:O;आजाद:O; -:O;प्रशांत:B-PER;पोळ:I-PER


मप्र:B-LOC;हाईकोर्ट:I-LOC;ने:O;भोपाल:B-LOC;के:O;लिए:O;निर्धारित:O;बाजार:O;मूल्य:O;की:O;गाइड-लाइन:O;के:O;उस:O;प्रावधान:O;पर:O;पूर्व:O;में:O;लगायी:O;गई:O;रोक:O;बरकरार:O;रखी:O;है:O;,:O;जिसके:O;तहत:O;फ्लैटों:O;की:O;बाजार:O;मूल्य:O;की:O;गणना:O;बिल्ट-अप:O;एरिया:O;व:O;सामान्य:O;सुविधाओं:O;को:O;शामिल:O;करते:O;हुए:O;की:O;जानी:O;थी:O;।:O


जब:O;मोदी:B-PER;मजबूत:O;होंगे:O;तो:O;देश:O;भी:O;और:O;मजबूत:O;होगा:O;।:O


इस:O;आम:O;चुनाव:O;में:O;भाजपा:B-ORG;नेता:O;सतीश:B-PER;कुमार:I-PER;गौतम:I-PER;को:O;सबसे:O;अधिक:O;6:B-MISC;लाख:I-MISC;56:I-MISC;हजार:I-MISC;215:I-MISC;वोट:O;प्राप्त:O;हुए:O;।:O


.4):O;देश:O;में:O;सभी:O;प्रकार:O;के:O;छोटे-बड़े:O;कृषि:O;,:O;सेवा:O;,:O;व्यापार:O;,:O;उद्योग:O;का:O;पंजीकरण:O;तुरंत:O;अनिवार्य:O;होना:O;चाहिए:O;,:O;साथ:O;उनको:O;रु:O;1000:B-MISC;से:O;ऊपर:O;के:O;लेन:O;देन:O;को:O;ऑन:O;लाइन/:O;इलेक्ट्रॉनिक:O;माध्यम:O;से:O;करना:O;अनिवार्य:O;होना:O;चाहिये।:O


निर्भया:B-PER;के:O;परिजनों:O;ने:O;आवेदन:O;किया:O;है:O;कि:O;उन्हें:O;फांसी:O;दिए:O;जाने:O;के:O;समय:O;उस:O;स्थान:O;पर:O;मौजूद:O;रहने:O;की:O;इजाजत:O;दी:O;जाए।:O


कच्चे:O;माल:O;की:O;लागत:O;राजस्व:O;का:O;तकरीबन:O;60:B-MISC;प्रतिशत:O;होती:O;है:O;और:O;दिसंबर:B-MISC;से:O;अप्रैल:B-MISC;के:O;बीच:O;इसमें:O;25:O;प्रतिशत:O;तक:O;का:O;इजाफा:O;हो:O;चुका:O;है।:O


उनका:O;अखबार:O;बंद:O;भले:O;हो:O;गया:O;पर:O;भाषा:O;पर:O;सरकारी:O;मोहर:O;लग:O;चुकी:O;है।:O


इस:O;पर:O;सेना:O;की:O;14:B-ORG;राष्ट्रीय:I-ORG;राइफल्स:I-ORG;,:O;जम्मू-कश्मीर:B-LOC;पुलिस:O;के:O;स्पेशल:O;ऑपरेशन:O;ग्रुप:O;और:O;केंद्रीय:B-ORG;रिजर्व:B-ORG;पुलिस:B-ORG;बल:B-ORG;के:O;जवानों:O;ने:O;जंगलों:O;की:O;घेराबंदी:O;कर:O;तलाशी:O;अभियान:O;शुरू:O;किया।:O


.कट्टर:O;सेक्स:O;,:O;बिग:O;डिक:O;,:O;बड़ी:O;डिक्स:O;,:O;उसे:O;बिल्ली:O;गड़बड़:O;हो:O;रही:O;है:O;,:O;बड़े:O;स्तन:O;,:O;बिल्ली:O;,:O;उसे:O;बिल्ली:O;कुचल:O;,:O;उसे:O;बिल्ली:O;पर:O;,:O;उसे:O;बिल्ली:O;गड़बड़:O;हो:O;,:O;get:O;ur:O;pussy:O;wet:O;porn:O;,:O;सभी:O;उसे:O;pussi:O;अश्लील:O;,:O;लड़की:O;अपने:O;पैर:O;की:O;उंगलियों:O;चूसना:O;हो:O;जाता:O;है।:O


आज:O;की:O;तारीख:O;है:O;10:B-MISC;अगस्त:I-MISC;2018:I-MISC;तथा:O;आज:O;इस:O;वर्ष:O;का:O;222वां:B-MISC;दिन:O;है।:O;इन:O;222:B-MISC;दिनों:O;में:O;प्रारंग:B-ORG;अपने:O;परिवार:O;के:O;जौनपुरवासियों:O;तक:O;208:B-MISC;लेख:O;पहुंचा:O;चुका:O;है:O;तथा:O;यह:O;लेख:O;209वां:B-MISC;लेख:O;होगा।:O


वहीं:O;प्रबंधन:B-MISC;समिति:B-MISC;कुसुम:B-PER;चाचान:I-PER;कहतीं:O;हैं:O;कि:O;मरीजों:O;एवं:O;उनके:O;स्वजनों:O;को:O;निशुल्क:O;भोजन:O;कराकर:O;सुख:O;की:O;अनुभूति:O;होती:O;है।:O


स्थिर:O;दर:O;को:O;पांच:B-MISC;से:O;10:B-MISC;साल:O;के:O;लिए:O;निश्चित:O;रखा:O;जा:O;सकता:O;है:O;और:O;उसके:O;बाद:O;वह:O;परिवर्तनशील:O;ब्याज:O;दर:O;की:O;श्रेणी:O;में:O;आ:O;जाएगा।:O


इस:O;सर्वेक्षण:O;में:O;छोटे:O;स्तर:O;की:O;49:B-MISC;फीसदी:O;इकाइयां:O;,:O;35:B-MISC;फीसदी:O;मझोली:O;इकाइयां:O;और:O;16:B-MISC;फीसदी:O;बड़ी:O;इकाइयां:O;शामिल:O;हुईं।:O


सामाजिक:O;संस्थाओं:O;व:O;आप:B-ORG;कार्यकर्ताओं:O;ने:O;नगरपालिका:B-ORG;के:O;खिलाफ:O;जमकर:O;नारेबाजी:O;की:O;।:O


संघ:O;के:O;प्रांतीय:B-MISC;सचिव:B-MISC;एवं:O;कोषाध्यक्ष:B-MISC;रणदीप:B-PER;आर्य:I-PER;ने:O;बताया:O;कि:O;बिजली:O;विभाग:O;में:O;कभी:O;ट्रांसफार्मर:O;दिलवाने:O;के:O;नाम:O;पर:O;और:O;कभी:O;तेल:O;व:O;केबल:O;के:O;नाम:O;पर:O;अतिरिक्त:O;शुल्क:O;मांगा:O;जाता:O;है:O;और:O;किसान:O;को:O;मजबूरीवश:O;यह:O;शुल्क:O;देना:O;पड़ता:O;है:O;।:O


मुरादाबाद:B-LOC;/:O;सिपाही:O;ने:O;कैंटीन:O;में:O;खड़े:O;लोगों:O;को:O;डंडों:O;से:O;पीटा:O;,:O;रिश्वत:O;भी:O;मांगी:O


जी:O;हां:O;विश्व:O;का:O;पहला:O;वजाइना:B-MISC;म्यूजियम:B-MISC;लंदन:B-LOC;में:O;खुलने:O;जा:O;रहा:O;है।:O


टॉयलेट:B-MISC;एक:I-MISC;कथा:I-MISC;:O;बिलासपुर:B-LOC;की:O;लड़की:O;ने:O;शादी:O;से:O;इनकार:O;किया:O;तब:O;युवक:O;ने:O;बनवाया:O;टॉयलेट:O;,:O;दिल्ली:B-LOC;की:O;टीम:O;ने:O;द:O;|:O


राजीव:B-PER;गांधी:I-PER;उर्फ:O;बॉबी:B-PER;ने:O;बताया:O;कि:O;विरोध:O;पर:O;नगर:O;निगम:O;ने:O;टावर:O;की:O;स्वीकृति:O;रद्द:O;कर:O;दी:O;थी:O;।:O

'''
Indic_NER=[['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O'], ['O', 'O', 'B-LOC', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'B-PER', 'O'], ['B-ORG', 'I-ORG', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'B-ORG', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'B-ORG', 'O', 'O', 'I-ORG', 'I-ORG', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-ORG', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O'], ['B-PER', 'I-PER', 'O', 'B-PER', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]



**Preprocessing ground truth and Predicted Labels**

In [6]:
truth=[]
tokens=[]
ground_truth=ground_truth.split('\n')
for sentence in ground_truth:
    temp=sentence.split(';')
    temp2=[]
    temp3=[]
    for words in temp:
        temp1=words.split(':')
        if len(temp1)<2:continue
        temp2.append(temp1[1])
        temp3.append(temp1[0])
    if len(temp2)>0:
        truth.append(temp2)
    if len(temp3)>0:tokens.append(temp3)
# for sentence in Indic_NER:
#     for i in range(len(sentence)):
#         if(sentence[i]=='B-LOC'): sentence[i]='O'
#         if(sentence[i]=='B-ORG'): sentence[i]='B-PER'
#         if(sentence[i]=='B-PER'): sentence[i]='I-PER'
#         if(sentence[i]=='I-LOC'): sentence[i]='B-ORG'
#         if(sentence[i]=='I-PER'): sentence[i]='B-LOC'
#         if(sentence[i]=='I-ORG'): sentence[i]='I-LOC' 

**Calculating Metrics**

In [9]:
def calculate_metrics(ground_truth, tokenizer_output):
    true_positives=[0 for i in range(9)]
    false_positives=[0 for i in range(9)]
    false_negatives=[0 for i in range(9)]
    for i in range (0, len(ground_truth)):
        for j in range(len(ground_truth[i])):
            if ground_truth[i][j]==tokenizer_output[i][j]:
                if ground_truth[i][j]=='B-PER':true_positives[0]+=1
                elif ground_truth[i][j]=='I-PER':true_positives[1]+=1
                elif ground_truth[i][j]=='B-LOC':true_positives[2]+=1
                elif ground_truth[i][j]=='I-LOC':true_positives[3]+=1
                elif ground_truth[i][j]=='B-ORG':true_positives[4]+=1
                elif ground_truth[i][j]=='I-ORG':true_positives[5]+=1
                elif ground_truth[i][j]=='B-MISC':true_positives[6]+=1
                elif ground_truth[i][j]=='I-MISC':true_positives[7]+=1
                else:true_positives[8]+=1
            else:
                if ground_truth[i][j]=='B-PER':false_positives[0]+=1
                elif ground_truth[i][j]=='I-PER':false_positives[1]+=1
                elif ground_truth[i][j]=='B-LOC':false_positives[2]+=1
                elif ground_truth[i][j]=='I-LOC':false_positives[3]+=1
                elif ground_truth[i][j]=='B-ORG':false_positives[4]+=1
                elif ground_truth[i][j]=='I-ORG':false_positives[5]+=1
                elif ground_truth[i][j]=='B-MISC':false_positives[6]+=1
                elif ground_truth[i][j]=='I-MISC':false_positives[7]+=1
                else:false_positives[8]+=1

                if ground_truth[i][j]=='B-PER':false_negatives[0]+=1
                elif ground_truth[i][j]=='I-PER':false_negatives[1]+=1
                elif ground_truth[i][j]=='B-LOC':false_negatives[2]+=1
                elif ground_truth[i][j]=='I-LOC':false_negatives[3]+=1
                elif ground_truth[i][j]=='B-ORG':false_negatives[4]+=1
                elif ground_truth[i][j]=='I-ORG':false_negatives[5]+=1
                elif ground_truth[i][j]=='B-MISC':false_negatives[6]+=1
                elif ground_truth[i][j]=='I-MISC':false_negatives[7]+=1
                else:false_negatives[8]+=1
                
    f_score=[0 for i in range(9)]
    precision=[0 for i in range(9)]
    recall=[0 for i in range(9)]
    for i in range(9):
        t_p=true_positives[i]
        f_p=false_positives[i]
        f_n=false_negatives[i]    
        if t_p + f_p != 0:
            precision[i] = t_p / (t_p + f_p) 
        else: precision[i] = 0
        if t_p + f_n != 0:
            recall[i] = t_p / (t_p + f_n)  
        else:recall[i] =  0
        if precision[i] + recall[i] != 0:
            f_score[i] = 2 * (precision[i] * recall[i]) / (precision[i] + recall[i])  
        else:f_score[i] =  0
    return f_score,precision,recall

**Metrics for each class and Macro_F1-Score**

In [10]:
fscore_NER,precision_NER,recall_NER=calculate_metrics(truth,Indic_NER)
tags=['B-PER','I-PER','B-LOC','I-LOC','B-ORG','I-ORG','B-MISC','I-MISC','Other']
for i in range(9):
    print("Precision of ",tags[i]," - ",precision_NER[i])
    print("Recall of ",tags[i]," - ",recall_NER[i])
    print("F-Score of ",tags[i]," - ",fscore_NER[i])
Macro_F1_Score=0
for i in fscore_NER:
    Macro_F1_Score+=i
Macro_F1_Score=Macro_F1_Score/9
print("Macro_F1_Score - ",Macro_F1_Score)

Precision of  B-PER  -  0.8181818181818182
Recall of  B-PER  -  0.8181818181818182
F-Score of  B-PER  -  0.8181818181818182
Precision of  I-PER  -  0.6666666666666666
Recall of  I-PER  -  0.6666666666666666
F-Score of  I-PER  -  0.6666666666666666
Precision of  B-LOC  -  0.6
Recall of  B-LOC  -  0.6
F-Score of  B-LOC  -  0.6
Precision of  I-LOC  -  0.0
Recall of  I-LOC  -  0.0
F-Score of  I-LOC  -  0
Precision of  B-ORG  -  0.36363636363636365
Recall of  B-ORG  -  0.36363636363636365
F-Score of  B-ORG  -  0.36363636363636365
Precision of  I-ORG  -  0.6666666666666666
Recall of  I-ORG  -  0.6666666666666666
F-Score of  I-ORG  -  0.6666666666666666
Precision of  B-MISC  -  0.0
Recall of  B-MISC  -  0.0
F-Score of  B-MISC  -  0
Precision of  I-MISC  -  0.0
Recall of  I-MISC  -  0.0
F-Score of  I-MISC  -  0
Precision of  Other  -  0.9607476635514018
Recall of  Other  -  0.9607476635514018
F-Score of  Other  -  0.9607476635514018
Macro_F1_Score -  0.45287768652254634


**Calculating Overall Metrics**

In [11]:
def calculate_metrics(ground_truth, tokenizer_output):
    t_p=0.0
    f_p=0.0
    f_n=0.0
    for i in range (0, len(ground_truth)):
        t_p+=len(set(ground_truth[i]) & set(tokenizer_output[i]))
        f_p+=len(set(tokenizer_output[i]) - set(ground_truth[i]))
        f_n+=len(set(ground_truth[i]) - set(tokenizer_output[i]))
    if t_p + f_p != 0:
        precision = t_p / (t_p + f_p) 
    else: precision = 0
    if t_p + f_n != 0:
        recall = t_p / (t_p + f_n)  
    else:recall =  0
    if precision + recall != 0:
        f_score = 2 * (precision * recall) / (precision + recall)  
    else:f_score =  0
    return precision, recall, f_score

In [12]:
precision,recall,fscore=calculate_metrics(truth,Indic_NER)
print("Precision - ",precision)
print("Recall - ",recall)
print("F-Score - ",fscore)

Precision -  0.8245614035087719
Recall -  0.6351351351351351
F-Score -  0.7175572519083969
